# Машинное обучение в бизнесе. Проектная работа. Выбор оптимального региона для бурения новых скважин.

## Описание проекта.

Добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Шаги для выбора локации :
* В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
* Строят модель для предсказания объёма запасов в новых скважинах;
* Выбирают скважины с самыми высокими оценками значений;
* Определяют регион с максимальной суммарной прибылью отобранных скважин.

Мы располагаем пробами нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Необходимо построит модель для определения региона, где добыча принесёт наибольшую прибыль, проанализировать возможную прибыль и риски техникой Bootstrap.

Условия задачи:
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Шаг 1. Загрузка и подготовька данных. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

Откроем файлы с данными.

In [ ]:
try:
    region_1 = pd.read_csv('geo_data_0.csv')
    region_2 = pd.read_csv('geo_data_1.csv')
    region_3 = pd.read_csv('geo_data_2.csv')
except:
    region_1 = pd.read_csv('datasets/geo_data_0.csv')
    region_2 = pd.read_csv('datasets/geo_data_1.csv')
    region_3 = pd.read_csv('datasets/geo_data_2.csv')

Так как структура датасетов одинакова, создадим функцию для описания данных, где просмотрим первые строки, получим информацию о структуре, описательную статистику и сведения о дубликатах.

In [ ]:
def data_description(data):
    display(data.head())
    
    display(data.info())
    
    display(data.describe(include='all').T)
    
    display('Дубликаты', data[data.duplicated()])
    print('-'*100)

Опишем данные, переберем датафреймы в цикле.

In [ ]:
for df in [region_1, region_2, region_3]:
    data_description(df)

Описание данных:
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

Пропусков и явных дубликатов в данных нет.

Но встречаются одинаковые значения id, но под ними скрываются скважины с разными характеристиками.

Возможно, это следствие синтетической природы данных, случайное совпадение, что id одинаковые.

Также, сравнив описательную статистику, видим, что первый и третий датасет похожи - средний запас в скважине и стандартное отклонение находятся приблизительно на одном уровне.

Создадим функцию для визуализации распределения признаков и целевого признака по всем трем датасетам, чтобы посмотреть, как они отличаются между собой, сведем все графики в один рисунок.


In [ ]:
def histograms_all(df_0,df_1,df_2):
    
    fig, axes = plt.subplots(3,4,figsize=(16,10))   
    
    columns_to_plot = df_0.columns[1:5]
    
    for i in range(len(columns_to_plot)):
        sns.distplot(df_0[columns_to_plot[i]], ax=axes[0, i], color='r')  
        
    for i in range(len(columns_to_plot)):
        sns.distplot(df_1[columns_to_plot[i]], ax=axes[1, i], color='g')
        
    for i in range(len(columns_to_plot)):
        sns.distplot(df_2[columns_to_plot[i]], ax=axes[2, i], color='b')
        
sns.set_style('whitegrid')
histograms_all(region_1, region_2, region_3)

Что же мы видим? Распределение запасов в скважине первого и третьего региона и правда достаточно похожи, при том, что у первого региона только признак f2 распределен нормально, а у третьего - все три.

Еще интереснее выглядит второй датасет. Видно, что признак f2 и целевой признак принимают только отдельные значения. Вероятно, они обладают очень сильной корреляцией. Трудно сказать, насколько такое явление может иметь место при разработке реальных скважин, но поскольку данные синтетические, то примем это как данность.

Для того, чтобы оценить корреляцию признаков и запасов в скважине - измерим ее и построим тепловые карты. Для этого создадим функцию.


In [ ]:
def heat_map(df,df_name):
    df = df.drop('id', axis=1)
    corr = df.corr(method='pearson')
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    f, ax = plt.subplots(figsize=(9, 6))
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', linewidths=0.5)
    plt.title(df_name)

In [ ]:
for k,df in {'region_1': region_1, 'region_2': region_2, 'region_3': region_1}.items():
    heat_map(df, k)

Получаем, что признаки f0 и f1 не имеют связи с целевым признаком по всем датасетам.

Есть связь запасов только с признаком f2, для первого и третьего набора данных коэффициент корреляции около 0.5, для второго, как и предполагалось выше - единица.

По всей видимости, такая связь f2 и запасов позволит нам построить очень точную модель линейной регрессии для второго региона.

### Вывод: 
Набор данных корректный во всех трех файлах, есть одинаковые значения id, но скважины явно разные, оставим, как есть.
На графиках выявили похожесть распределения запасов в скважинах первого и третьего региона. А также корреляцию признаков и запасов во втором регионе, что подтвердилось тепловыми картами.

## Шаг 2. Обучение и проверка модели для каждого региона.

Создадим функцию, в которой будем вызывать train_test_split для разбивки данных на обучающую и валидационную выборки в соотношении 75:25


In [ ]:
def split_for_train_valid(df):
    target = df['product']
    features = df.drop(['id','product'], axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                test_size=0.25, random_state=1)
    return features_train, features_valid, target_train, target_valid

После того, как мы обучим модели и сделаем предсказания на валидационной выборке, нам потребуется сохранить предсказания и ответы на валидационной выборке, а также вывести на экран средний предсказанный объем запасов и корень из средней квадратичной ошибки.

Для этих целей также напишем функцию, внутри которой будет проходить обучение модели, предсказание и расчет необходимых метрик.

Дополнительно рассчитаем R2.

In [ ]:
def predicted_mean_and_rmse(arrays):
    model = LinearRegression()
    model = model.fit(arrays[0], arrays[2]) # индексы соответствуют порядку в split_for_train_valid
    predictions_valid = model.predict(arrays[1])
    
    rmse = mean_squared_error(arrays[3], predictions_valid) ** 0.5
    mean_prediction = predictions_valid.mean()
    R2 = model.score(arrays[1], arrays[3])
    
    return predictions_valid, arrays[3], mean_prediction, rmse, R2, model

Далее создаём нужные нам переменные для каждого региона, в которые присваиваем результат вызова predicted_mean_and_rmse.

В predicted_mean_and_rmse, в виде аргумента будет передаваться функция split_for_train_valid, в которую, в свою очередь, в виде аргумента будет передаваться нужный нам датафрейм.

In [ ]:
predictions_valid_1, target_valid_1, mean_product_1, rmse_1, R2_1, model_1 = \
                                            predicted_mean_and_rmse(split_for_train_valid(region_1))

predictions_valid_2, target_valid_2, mean_product_2, rmse_2, R2_2, model_2 = \
                                            predicted_mean_and_rmse(split_for_train_valid(region_2))

predictions_valid_3, target_valid_3, mean_product_3, rmse_3, R2_3, model_3 = \
                                            predicted_mean_and_rmse(split_for_train_valid(region_3))

Выведем на экран интересующие нас результаты. Создадим словарь для удобства.

In [ ]:
regions = {'Первый регион' : [mean_product_1, rmse_1, R2_1],
           'Второй регион' : [mean_product_2, rmse_2, R2_2],
           'Третий регион' : [mean_product_3, rmse_3, R2_3]}

In [ ]:
for region, values in regions.items():
    print(region)
    print(f'Средний запас предсказанного сырья: {values[0]:.2f} тыс. баррелей')
    print(f'RMSE: {values[1]:.2f} тыс. баррелей')
    print(f'R2: {values[2]:.2f}')
    print('')

### Вывод: 

Средний предсказанный запас сырья в скважине в первом и третьем регионе находится примерно на одном уровне. В то время как во втором регионе он более чем на 20 тыс. баррелей ниже.

У предсказаний по первому и третьему региону велик корень из средней квадратичной ошибки - почти 40 тыс. баррелей.

Второй регион, данные по которому обладали признаком, имеющим очень сильную связь с запасами в скважине, показал незначительный RMSE.

Как следствие, коэффициенты детерминации для моделей первого и третьего регионов говорят о том, что дисперсия запасов в скважине только на 28% и 20% соответственно объясняется признаками f0, f1, f2.

Это означает, что модели для первого и третьего регионов обладают достаточно низким качеством, в то время как модель для второго региона предсказывает запасы почти идеально.



## Шаг 3. Подготовка к расчёту прибыли.

Все рассчеты будем производить в рублях и баррелях, а не в тыс. рублей и тыс. баррелей, если не будет отмечено отдельно.

Итак, бюджет для разработки скважин составляет 10 млрд руб.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

Нам нужно отобрать 200 лучших скважин, отдельно запишем это значение в переменную.

In [ ]:
total_budget = 10000000000
revenue_per_barrel = 450
wells_for_drilling = 200

Посчитаем, сколько нужно потратить на бурение одной скважины.

Делая рассчёт, мы предполагаем о том, что скважины все относительно стандартные и бюджет для бурения каждой - одинаков. В любом случае, условий задачи недостаточно, чтобы сделать другие выводы.

In [ ]:
investments_per_well = total_budget / wells_for_drilling
print(f'{investments_per_well:.0f}')

Посчитаем, сколько сырья должно быть в скважине, чтобы ее разработка была безубыточной.

Разделим бюджет для скважины на выручку с барреля.

In [ ]:
breakeven_reserves_per_well = investments_per_well / revenue_per_barrel
print(f'{breakeven_reserves_per_well:.0f}')

### Вывод: 
Получается, что 50 млн руб. - бюджет для разработки одной скважины.
В каждом регионе только около 20% скважин, согласно предсказаниям моделей, превышают безубыточный уровень.

# Шаг 4. Расчёт прибыли и рисков.

Создадим функцию расчета прибыли.
Функция для расчета прибыли должна выбирать скважины с наибольшей величиной предсказанных запасов, и на основе ответов посчитать выручку.

In [ ]:
def profit(wells_target, wells_predicted, number_of_wells_to_pick):
    
    wells_predicted_sorted = pd.Series(wells_predicted).sort_values(ascending=False)
    wells_picked = wells_target[wells_predicted_sorted.index][:number_of_wells_to_pick]
    wells_picked_total_product = wells_picked.sum() * 1000
    
    profit = wells_picked_total_product * revenue_per_barrel - total_budget
    return profit

Создадим функцию и операции бутстреп, чтобы потом применять ее для регионов.

Внутри нее рассчитаем 95% доверительный интервал и риск убытков.

Риск убытков рассчитаем функцией модуля scipy.stats.percentileofscore, которая покажает, какой процент значений массива с прибылями лежит ниже нуля. Это для нас и будет выражение риска убытков.

Также внутри функции перезагрузим индексы для ответов, поскольку они принадлежат интервалу от 0 до 100000 случайным образом, а индексы предсказаний идут по порядку от 0 до 24999.

In [ ]:
def profit_bootstrap(target, predictions, number_of_wells_to_observe, number_of_wells_to_pick):
    
    state = np.random.RandomState(17)
    profits = []
    predictions = pd.Series(predictions)
    
    target_new_index = target.reset_index(drop=True)
    
    for i in range(1000):
        target_subsample = target_new_index.sample(n=number_of_wells_to_observe, replace=True, 
                                               random_state=state)
        predictions_subsample = predictions[target_subsample.index]
        profits.append(profit(target_subsample, predictions_subsample, number_of_wells_to_pick))
        
    profits = pd.Series(profits)
    mean_profit = profits.mean()
    
    lower = profits.quantile(0.025)
    upper = profits.quantile(0.975)
    
    loss_risk = st.percentileofscore(profits, 0)  #какой процент значений ниже нуля
    
    return profits, mean_profit, lower, upper, loss_risk

Присвоим результаты вызова функции в интересующие нас переменные.

In [ ]:
profits_1, mean_profit_1, lower_1, upper_1, loss_risk_1 = profit_bootstrap(target_valid_1,
                                                                    predictions_valid_1,500, 200)

profits_2, mean_profit_2, lower_2, upper_2, loss_risk_2 = profit_bootstrap(target_valid_2,
                                                                    predictions_valid_2, 500, 200)

profits_3, mean_profit_3, lower_3, upper_3, loss_risk_3 = profit_bootstrap(target_valid_3,
                                                                    predictions_valid_3, 500, 200)

Создадим словарь для перебора параметров.

In [ ]:
profit_metrics = {'Первый регион' : [mean_profit_1, lower_1, upper_1, loss_risk_1],
           'Второй регион' : [mean_profit_2, lower_2, upper_2, loss_risk_2],
           'Третий регион' : [mean_profit_3, lower_3, upper_3, loss_risk_3]}

In [ ]:
for region, values in profit_metrics.items():
    print(region)
    print(f'Средняя прибыль: {values[0] / 1000000:.0f} млн руб.')
    print(f'95% доверительный интервал для прибыли:({values[1] / 1000000:.0f},{values[2] / 1000000:.0f}) млн руб.')
    print(f'Риск убытков: {values[3]}%')
    print('')

### Вывод
Итак, риск убытков выше 2.5% в первом и третьем регионе. Значит, по этому параметру отбора региона для разработки они не подходят.

Остается второй регион, где риск убытков составляет 1%, да и значение средней прибыли выше, чем в остальных.

Наверное, этот регион отобран "по праву", поскольку модель предсказания запасов для его скважин выглядит самой качественной, оценивая параметр R2, принимая как данность тот факт, что для этого региона значения параметра f2 и запасов обладают ярко выраженной синтетичностью.